In [33]:
import pandas as pd

satış_datası = pd.read_excel("Satış Datası.xlsx")
print(satış_datası)


#########Toplam satış tutarı###############################
toplam_fiyat = 0
urun_tutarları = satış_datası["URUN_TUTARI"]
urun_adetleri = satış_datası["URUN_ADEDI"]
for i in range(0, len(urun_adetleri)):
    toplam_fiyat += urun_tutarları[i] * urun_adetleri[i]
    
print("Toplam satış tutarı: "+str(toplam_fiyat))
print((urun_tutarları*urun_adetleri).sum())
#########Toplam satış tutarı###############################


#########Toplam fiş sayısı ve ürün adedi##################################
fişler = satış_datası["FIS_ID"]
print()
value_counts = fişler.value_counts()
print(value_counts)

toplam_fiş_sayısı = 0
toplam_urun_adedi = 0
for count in value_counts:
    toplam_fiş_sayısı += count
    
for adet in urun_adetleri:
    toplam_urun_adedi += adet

print("Toplam fiş sayısı:", toplam_fiş_sayısı)
print("Toplam ürün adedi:", toplam_urun_adedi)
#########Toplam fiş sayısı ve ürün adedi##################################


# b. Gün bazında toplam fiş sayısı, toplam satış tutarı ve ortalama sepet tutarı
satış_datası['ALISVERIS_TARIHI'] = pd.to_datetime(satış_datası['ALISVERIS_TARIHI'], format='%Y%m%d')
gunlere_gore_satis = satış_datası.groupby('ALISVERIS_TARIHI').agg(
    ToplamFisSayisi=('FIS_ID', 'nunique'),
    ToplamSatisTutari=('URUN_TUTARI', lambda x: (x * satış_datası.loc[x.index, 'URUN_ADEDI']).sum())
) 
gunlere_gore_satis["OrtalamaSepetTutarı"] = gunlere_gore_satis["ToplamSatisTutari"] / gunlere_gore_satis["ToplamFisSayisi"]

print(gunlere_gore_satis)
    
    
formatlara_gore_fis_tutarlari = satış_datası.groupby('MAGAZA_FORMAT_KODU').apply(lambda group: (group['URUN_TUTARI'] * group['URUN_ADEDI']).describe())
print(formatlara_gore_fis_tutarlari)


###print(formatlara_gore_fis_tutarlari[['min', 'mean', '50%', 'max']])
    

        ALISVERIS_TARIHI  MAGAZA_KODU MAGAZA_FORMAT_KODU  \
0               20190901         1019              HIPER   
1               20190901         3472              SUPER   
2               20190908         1004              HIPER   
3               20190901         1004              HIPER   
4               20190908         1004              HIPER   
...                  ...          ...                ...   
151210          20190909         3213              SUPER   
151211          20190909         3035              SUPER   
151212          20190909         3062              SUPER   
151213          20190909         3324              SUPER   
151214          20190909         3569              SUPER   

                        FIS_ID  URUN_KODU  URUN_TUTARI  URUN_ADEDI  
0        201909010000001019385   30060124         0.25       1.000  
1       2019090100000034721130   30060124         0.25       1.000  
2       2019090800000010043989   30060124         0.25       1.000  
3  

In [22]:
import pandas as pd

urun_kategori_verisi = pd.read_excel("UrunKategori.xlsx")
print(urun_kategori_verisi)

         URUN_KODU                           URUN_ISMI  MALGRUBU_NO  \
0         30204315    MONIN CHOCOLATE ŞURUP 70 CL ŞİŞE    100000001   
1         30204316      MONIN VANİLYA ŞURUP 70 CL ŞİŞE    100000001   
2         30204312      MONIN CARAMEL ŞURUP 70 CL ŞİŞE    100000001   
3         30204189      MONIN COCONUT ŞURUP 70 CL ŞİŞE    100000001   
4         30204314  MONIN MOJİTO MINT ŞURUP 70 CL ŞİŞE    100000001   
...            ...                                 ...          ...   
41428     30192917           PRATİK PAÇA YAPIŞTIRICISI    605080300   
41429  30066893001        PERDE STOPU DURDUR, ASS, STD    605080300   
41430  30066737001               BEBE LASTİK, ASS, STD    605080500   
41431  30066764001         YASSI LASTİK NO:2, ASS, STD    605080500   
41432     40006637            E-TİCARET NAKLİYE BEDELİ    802050201   

             MALGRUBU_ADI  AILE_NO      AILE_ADI  GRANDAILE_NO  \
0      DIGER MEYVE SURUBU  1000000         SURUP         10000   
1      DIGER ME

In [32]:
# Join the data frames on common keys
joined_data = urun_kategori_verisi.merge(satış_datası, on='URUN_KODU')

# Group by 'REYON_ADI' to get total fiş sayısı and total harcama tutarı for each reyon
reyon_bazinda_analiz = joined_data.groupby('REYON_ADI').agg(
    ToplamFisSayisi=('FIS_ID', 'nunique'),
    ToplamHarcamaTutari=('URUN_TUTARI', lambda x: (x * joined_data.loc[x.index, 'URUN_ADEDI']).sum())
)

print(reyon_bazinda_analiz)
print()
print()

# Find the reyon with the highest ciro pay
en_yuksek_ciro_reyon = reyon_bazinda_analiz['ToplamHarcamaTutari'].idxmax()
print("En yüksek ciro reyon "+str(en_yuksek_ciro_reyon))

# Filter the data for the reyon with the highest ciro pay
highest_ciro_reyon_data = joined_data[joined_data['REYON_ADI'] == en_yuksek_ciro_reyon]

# Group by 'AILE_ADI' to get total fiş sayısı and total harcama tutarı for each aile grupu within the reyon
aile_grubu_bazinda_analiz = highest_ciro_reyon_data.groupby('AILE_ADI').agg(
    ToplamFisSayisi=('FIS_ID', 'nunique'),
    ToplamHarcamaTutari=('URUN_TUTARI', lambda x: (x * highest_ciro_reyon_data.loc[x.index, 'URUN_ADEDI']).sum())
)
print()
print()
print(aile_grubu_bazinda_analiz)


# Find the aile grupu with the highest ciro pay
en_yuksek_ciro_aile = aile_grubu_bazinda_analiz['ToplamHarcamaTutari'].idxmax()

# Filter the data for the aile grupu with the highest ciro pay within the reyon
highest_ciro_aile_data = highest_ciro_reyon_data[highest_ciro_reyon_data['AILE_ADI'] == en_yuksek_ciro_aile]

# Calculate the total ciro for the aile grupu (family group) considering quantity
total_aile_ciro = (highest_ciro_aile_data['URUN_TUTARI'] * highest_ciro_aile_data['URUN_ADEDI']).sum()

# Calculate ciro penetrasyonu for each ürün within the aile grupu
ciro_penetrasyonu = highest_ciro_aile_data.groupby('URUN_ISMI').apply(
    lambda group: (group['URUN_TUTARI'] * group['URUN_ADEDI']).sum() / total_aile_ciro
)

# Calculate total number of unique receipts for the highest revenue Aile Grubu
total_aile_receipts = highest_ciro_aile_data['FIS_ID'].nunique()

# Calculate total number of unique receipts for each product in the highest revenue Aile Grubu
product_receipt_counts = highest_ciro_aile_data.groupby('URUN_ISMI')['FIS_ID'].nunique()

# Calculate Basket Penetrasyonu for each product
basket_penetrasyonu = product_receipt_counts / total_aile_receipts


print()
print()

print(ciro_penetrasyonu)

print()
print()

print(basket_penetrasyonu)




                      ToplamFisSayisi  ToplamHarcamaTutari
REYON_ADI                                                 
ALTIN-OR                            1         2.980000e+02
AYAKKABI                          611         1.481894e+04
BAHÇECILIK                        731         2.919871e+04
BALIK                             574         1.189637e+06
BAYAN                              82         1.856330e+03
BILGISAYAR                         57         7.023040e+03
BÜYÜK BEYAZ ESYALAR                12         1.749505e+04
ERKEK                             106         6.172500e+03
EV GEREÇLERI                     3285         1.724521e+05
EVTEKSTIL                         411         1.299969e+04
FOTO&BÜRO                         124         2.536764e+04
KASAP                            2011         1.200894e+05
KURU GIDALAR                    13977         5.537893e+05
KÜLTÜR                           3489         2.708460e+05
KÜÇÜK BEYAZ ESYALAR               121         2.505520e+